In [1]:
!git clone https://github.com/hkproj/pytorch-stable-diffusion.git
!git clone https://huggingface.co/pt-sk/SDcheckpoint
!git clone https://huggingface.co/pt-sk/CLIPTOKENIZER

Cloning into 'pytorch-stable-diffusion'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 69 (delta 21), reused 14 (delta 14), pack-reused 36
Unpacking objects: 100% (69/69), 1.96 MiB | 6.59 MiB/s, done.
Cloning into 'SDcheckpoint'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 7 (delta 0), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (7/7), 2.20 KiB | 2.20 MiB/s, done.
Encountered 1 file(s) that may not have been copied correctly on Windows:
	v1-5-pruned-emaonly.ckpt

See: `git lfs help smudge` for more details.


In [3]:
import os
from shutil import copyfile
files = os.listdir("/kaggle/working/pytorch-stable-diffusion/sd")

for file in files:
    if file.endswith(".py"): 
        copyfile(src = f"/kaggle/working/pytorch-stable-diffusion/sd/{file}", dst = f"/kaggle/working/{file}")

In [11]:
# necessary libraries
import math
import json
import pandas as pd
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
from tqdm import tqdm
from torch.optim import AdamW
from torchvision import transforms
from transformers import CLIPTokenizer

import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

from diffusion import Diffusion
from model_loader import preload_models_from_standard_weights
from ddpm import DDPMSampler
# import torch_xla
# import torch_xla.core.xla_model as xm

In [5]:
# CONSTANT VARIABLES
BASE_PATH = '../input/coco-2017-dataset/coco2017'
IMAGES = ["train2017", "val2017", "test2017"]
IMAGE_SIZE = (512, 512)
NUM_WORKERS = 4
BATCH_SIZE = 2
# device = xm.xla_device()

In [6]:
# using train2017
with open(f'{BASE_PATH}/annotations/captions_train2017.json', 'r') as f:
    data = json.load(f)
    data = data['annotations']

img_cap_pairs = []

for sample in data:
    img_name = '%012d.jpg' % sample['image_id']
    img_cap_pairs.append([img_name, sample['caption']])

captions = pd.DataFrame(img_cap_pairs, columns=['image', 'caption'])
captions['image'] = captions['image'].apply(
    lambda x: f'{BASE_PATH}/train2017/{x}'
)
captions = captions.head(50000)
captions.head()

,image,caption
0,../input/coco-2017-dataset/coco2017/train2017/...,A bicycle replica with a clock as the front wh...
1,../input/coco-2017-dataset/coco2017/train2017/...,A room with blue walls and a white sink and door.
2,../input/coco-2017-dataset/coco2017/train2017/...,A car that seems to be parked illegally behind...
3,../input/coco-2017-dataset/coco2017/train2017/...,A large passenger airplane flying through the ...
4,../input/coco-2017-dataset/coco2017/train2017/...,There is a GOL plane taking off in a partly cl...


In [7]:
# transformations for the images
transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE, transforms.InterpolationMode.BICUBIC),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor()
])

In [8]:
# creating a customimage dataset
class ImageCaptions_Dataset(Dataset):
    def __init__(self, captions: pd.DataFrame, transforms):
        self.dataframe = captions
        self.transforms = transforms
        
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        image = self.dataframe.iloc[idx]["image"]
        caption = self.dataframe.iloc[idx]["caption"]
        
        image = Image.open(image)
        image = self.transforms(image)
        
        return image, caption

# creating image captions dataset
image_captions_dataset = ImageCaptions_Dataset(captions, transform)

# creating image captions dataloader
image_captions_dataloader = DataLoader(image_captions_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [9]:
# instantiating models
models = preload_models_from_standard_weights("/kaggle/working/SDcheckpoint/v1-5-pruned-emaonly.ckpt", "cuda:1")
encoder = models["encoder"]
decoder = models["decoder"]
clip = models["clip"]

model = Diffusion().to("cuda:0")

In [10]:
a, b = next(iter(image_captions_dataloader))

In [13]:
tokenizer = CLIPTokenizer("/kaggle/working/CLIPTOKENIZER/vocab.json", merges_file="/kaggle/working/CLIPTOKENIZER/merges.txt")

In [21]:
text = tokenizer.batch_encode_plus(
                b, padding="max_length", max_length=77
            ).input_ids